# Data Science u oblaku: Način "Azure ML SDK"

## Uvod

U ovom ćemo bilježniku naučiti kako koristiti Azure ML SDK za treniranje, implementaciju i korištenje modela putem Azure ML-a.

Preduvjeti:
1. Kreirali ste Azure ML radni prostor.
2. Učitali ste [Heart Failure dataset](https://www.kaggle.com/andrewmvd/heart-failure-clinical-data) u Azure ML.
3. Prenijeli ste ovaj bilježnik u Azure ML Studio.

Sljedeći koraci su:

1. Kreirajte Eksperiment u postojećem Radnom prostoru.
2. Kreirajte Compute klaster.
3. Učitajte skup podataka.
4. Konfigurirajte AutoML koristeći AutoMLConfig.
5. Pokrenite AutoML eksperiment.
6. Istražite rezultate i pronađite najbolji model.
7. Registrirajte najbolji model.
8. Implementirajte najbolji model.
9. Koristite krajnju točku.

## Specifični uvozi za Azure Machine Learning SDK


In [ ]:
from azureml.core import Workspace, Experiment
from azureml.core.compute import AmlCompute
from azureml.train.automl import AutoMLConfig
from azureml.widgets import RunDetails
from azureml.core.model import InferenceConfig, Model
from azureml.core.webservice import AciWebservice

## Inicijalizacija radnog prostora
Inicijalizirajte objekt radnog prostora iz spremljene konfiguracije. Provjerite je li konfiguracijska datoteka prisutna na .\config.json


In [ ]:
ws = Workspace.from_config()
print(ws.name, ws.resource_group, ws.location, ws.subscription_id, sep = '\n')

## Kreirajte Azure ML eksperiment

Kreirajmo eksperiment pod nazivom 'aml-experiment' u radnom prostoru koji smo upravo inicijalizirali.


In [ ]:
experiment_name = 'aml-experiment'
experiment = Experiment(ws, experiment_name)
experiment

## Kreirajte klaster za računalnu obradu
Morat ćete kreirati [cilj za računalnu obradu](https://docs.microsoft.com/azure/machine-learning/concept-azure-machine-learning-architecture#compute-target) za vaše AutoML pokretanje.


In [ ]:
aml_name = "heart-f-cluster"
try:
    aml_compute = AmlCompute(ws, aml_name)
    print('Found existing AML compute context.')
except:
    print('Creating new AML compute context.')
    aml_config = AmlCompute.provisioning_configuration(vm_size = "Standard_D2_v2", min_nodes=1, max_nodes=3)
    aml_compute = AmlCompute.create(ws, name = aml_name, provisioning_configuration = aml_config)
    aml_compute.wait_for_completion(show_output = True)

cts = ws.compute_targets
compute_target = cts[aml_name]

## Podaci
Provjerite jeste li učitali skup podataka u Azure ML i da je ključ istog naziva kao i skup podataka.


In [ ]:
key = 'heart-failure-records'
dataset = ws.datasets[key]
df = dataset.to_pandas_dataframe()
df.describe()

## Konfiguracija AutoML


In [ ]:
automl_settings = {
    "experiment_timeout_minutes": 20,
    "max_concurrent_iterations": 3,
    "primary_metric" : 'AUC_weighted'
}

automl_config = AutoMLConfig(compute_target=compute_target,
                             task = "classification",
                             training_data=dataset,
                             label_column_name="DEATH_EVENT",
                             enable_early_stopping= True,
                             featurization= 'auto',
                             debug_log = "automl_errors.log",
                             **automl_settings
                            )

## AutoML Pokretanje


In [ ]:
remote_run = experiment.submit(automl_config)

In [ ]:
RunDetails(remote_run).show()

In [ ]:
best_run, fitted_model = remote_run.get_output()

In [ ]:
best_run.get_properties()

In [ ]:
model_name = best_run.properties['model_name']
script_file_name = 'inference/score.py'
best_run.download_file('outputs/scoring_file_v_1_0_0.py', 'inference/score.py')
description = "aml heart failure project sdk"
model = best_run.register_model(model_name = model_name,
                                description = description,
                                tags = None)

## Implementirajte najbolji model

Pokrenite sljedeći kod za implementaciju najboljeg modela. Status implementacije možete vidjeti u Azure ML portalu. Ovaj korak može potrajati nekoliko minuta.


In [ ]:
inference_config = InferenceConfig(entry_script=script_file_name, environment=best_run.get_environment())

aciconfig = AciWebservice.deploy_configuration(cpu_cores = 1,
                                               memory_gb = 1,
                                               tags = {'type': "automl-heart-failure-prediction"},
                                               description = 'Sample service for AutoML Heart Failure Prediction')

aci_service_name = 'automl-hf-sdk'
aci_service = Model.deploy(ws, aci_service_name, [model], inference_config, aciconfig)
aci_service.wait_for_deployment(True)
print(aci_service.state)

## Korištenje krajnje točke
Možete dodati ulaze u sljedeći primjer ulaza.


In [ ]:
data = {
    "data":
    [
        {
            'age': "60",
            'anaemia': "false",
            'creatinine_phosphokinase': "500",
            'diabetes': "false",
            'ejection_fraction': "38",
            'high_blood_pressure': "false",
            'platelets': "260000",
            'serum_creatinine': "1.40",
            'serum_sodium': "137",
            'sex': "false",
            'smoking': "false",
            'time': "130",
        },
    ],
}

test_sample = str.encode(json.dumps(data))

In [ ]:
response = aci_service.run(input_data=test_sample)
response


---

**Odricanje od odgovornosti**:  
Ovaj dokument je preveden pomoću AI usluge za prevođenje [Co-op Translator](https://github.com/Azure/co-op-translator). Iako nastojimo osigurati točnost, imajte na umu da automatski prijevodi mogu sadržavati pogreške ili netočnosti. Izvorni dokument na izvornom jeziku treba smatrati autoritativnim izvorom. Za ključne informacije preporučuje se profesionalni prijevod od strane čovjeka. Ne preuzimamo odgovornost za bilo kakva nesporazuma ili pogrešna tumačenja koja proizlaze iz korištenja ovog prijevoda.
